**Qual dos serviços mais falha? (status = PROVIDER_FAILED)**

In [0]:
with

  base as (
    select
      bt.provedor,
      count(*) filter(where bt.status = 'falha no provedor') qtd_execucoes_falhadas,
      count(*) qtd_execucoes
    from
      hive_metastore.gold.fato_biometria_tentativas bt
    group by
      1
  ),

  interv_confianca as (
    select
      b.provedor,
      b.qtd_execucoes_falhadas,
      b.qtd_execucoes,
      round(
        (b.qtd_execucoes_falhadas / b.qtd_execucoes),
        2
      ) tx_falha,
      qtd_execucoes_falhadas * 1.0 / qtd_execucoes p_hat,
      sqrt(
        (qtd_execucoes_falhadas * 1.0 / qtd_execucoes)
        * (1 - qtd_execucoes_falhadas * 1.0 / qtd_execucoes)
        / qtd_execucoes
      ) se
    from
      base b
  )

select
  ic.provedor,
  ic.qtd_execucoes_falhadas,
  ic.qtd_execucoes,
  round(
    p_hat - 1.96 * se,
    2
  ) AS ic_inferior,
  ic.tx_falha,
  round(
    p_hat + 1.96 * se,
    2
  ) AS ic_superior
from
  interv_confianca ic
order by
  1 asc

**Qual categoria de entregador possui o maior índice falha na identificação biométrica? (status =
NOT_MATCH)**

In [0]:
select
  b.status,
  m.nvl_categoria,
  count(*) qtd_execucoes_nao_combinam
from
  hive_metastore.gold.fato_biometria_tentativas bt
  left join
    hive_metastore.gold.dim_biometria b
    on
      bt.id_sessao = b.id_sessao
  left join
    hive_metastore.gold.dim_motoristas m
    on
      b.id_motorista = m.id_motorista
where
  b.status = 'nao combina'
group by
  1, 2
order by
  2

**Calcule quanto o índice geral de MATCH seria se aumentássemos a similaridade mínima do
MATCH para 0.90**

In [0]:
with 

  base as (
    select
      count(*) filter(where bt.status = 'combina') qtd_atual_combina,
      sum(bt.flg_similaridade_maior_90_percent) filter(where bt.flg_similaridade_maior_90_percent = 1) qtd_threshold_ajustada,
      count(*) qtd_total
    from
      hive_metastore.gold.fato_biometria_tentativas bt
  ),

  base_fim as (
    select
      b.*,
      round(
        (b.qtd_atual_combina * 1.0 / b.qtd_total),
        2
      ) qtd_atual_combina_tx,
      round(
        (b.qtd_threshold_ajustada * 1.0 / b.qtd_total),
        2
      ) qtd_threshold_ajustada_tx,
      round(
        (1-((qtd_threshold_ajustada / qtd_atual_combina) * 1.0)),
        2
      ) tx_reducao_atual,
      round(
        ((b.qtd_atual_combina - b.qtd_threshold_ajustada) * 1.0 / b.qtd_total),
        2
      ) tx_reducao_absoluta
    from
      base b 
  )

select
  bf.*
from
  base_fim bf


**Você diria que há alguma relação entre volume de pedidos cancelados (status = CANCELLED) de
um entregador e o status final do processo de identificação biométrica dele? Justifique sua
resposta.**

In [0]:
with 

  pedido_cancelado as (
    select
      os.id_motorista,
      count(*) filter (where os.status_ordem = 'cancelado') qtd_cancelado
    from
      hive_metastore.gold.fato_ordens_servico os
    group by
      1
  )

select
  b.status,
  round(
    avg(pc.qtd_cancelado),
    2
  ) media
from
  hive_metastore.gold.dim_biometria b
  left join
    pedido_cancelado pc
    on
      pc.id_motorista = b.id_motorista
where
  status in ("combina", "nao combina")
group by
  1

**Em quais dias provavelmente tivemos um aumento do comportamento indevido “empréstimo de
conta”?**

In [0]:
select
  date(date_trunc('day', dt_evento)) dt,
  count(*) filter(where status = 'nao combina') qtd_nao_combina
from
  hive_metastore.gold.fato_biometria_tentativas
group by
  1
order by
  1 asc